# Import Modules and Inputs

In [1]:
import ee, geemap, time

Intialize Geemap

In [2]:
Map = geemap.Map(center=(10.41, 37.62), zoom=11)

jedeb = ee.FeatureCollection("projects/ee-yilkalg3/assets/Jedeb_Watershed")
#Map.addLayer(jedeb, {}, 'Jedeb Watershed')

Set Dates

In [3]:
start = '2021-12-01'
end = '2022-04-30'
season = ee.Filter.date(start,end)

Import Training Dataset

In [12]:
points = ee.FeatureCollection("projects/ee-yilkalg3/assets/TrainingJ")
print(points.size().getInfo())

896


Import Validation Dataset

In [13]:
validationGcp = ee.FeatureCollection("projects/ee-yilkalg3/assets/ValidationJ")
print(validationGcp.size().getInfo())

225


In [14]:
#validationGcp.first().getInfo()

# Sentinel 2 only

In [15]:
# Load Median Sentinel 2 Reflectance 
def maskS2clouds(image):
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
      .And(qa.bitwiseAnd(cirrusBitMask).eq(0))

  return image.updateMask(mask).divide(10000)

# Filter the first Sentinel 2 image of the month
sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR')\
    .filterBounds(jedeb)\
    .filter(season)\
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20))\
    .map(maskS2clouds)
 
# Median image
s2median = ee.Image(sentinel2.median()).clip(jedeb)
s2median.bandNames().getInfo()

visualization = {
  'min': 0.0,
  'max': 0.3,
  'bands': ['B4', 'B3', 'B2'],
}

Map.addLayer(s2median, visualization, 'RGB')

## Train the classifier

In [16]:
# Use these bands for prediction (excluding red edge bands).
bands = ['B2', 'B3', 'B4', 'B5', 'B6','B7','B8', 'B8A', 'B11', 'B12']

# This property of the table stores the land cover labels.
label = 'Cover'

# Overlay the points on the imagery to get training.
sample = s2median.select(bands).sampleRegions(
    **{'collection': points, 'properties': [label], 'scale': 10}
)

classifier = ee.Classifier.smileCart(128).train(sample, label, bands)

## Classify the Image

In [17]:
# Classify the image with the same bands used for training 
classified_RF1 = s2median.select(bands).classify(classifier)

## Accuracy assessment

Training Accuracy

In [18]:
train_accuracy = classifier.confusionMatrix()
#train_accuracy.getInfo()

In [19]:
#train_accuracy.accuracy().getInfo()

In [20]:
#train_accuracy.kappa().getInfo()

In [21]:
#train_accuracy.producersAccuracy().getInfo()

In [22]:
#train_accuracy.consumersAccuracy().getInfo()

Validation Accuracy

In [23]:
# This property of the table stores the land cover labels.
label = 'Cover'

band = ['classification']
# Overlay the points on the imagery to get training.
test = classified_RF1.select(band).sampleRegions(
    **{'collection': validationGcp, 'properties':[label],'scale': 10}
)

In [24]:
test_accuracy = test.errorMatrix('Cover', 'classification')
#test_accuracy.getInfo()

In [25]:
test_accuracy.accuracy().getInfo()

0.72

In [26]:
test_accuracy.kappa().getInfo()

0.6422080872330759

In [27]:
test_accuracy.producersAccuracy().getInfo()

[[0],
 [1],
 [0.7571428571428571],
 [0.6470588235294118],
 [0.7272727272727273],
 [0.6792452830188679],
 [0.8333333333333334],
 [0.6666666666666666]]

# Sentinel 1 and Sentinel 2

In [4]:
# Import Sentinel-1 collection
sentinel1 =  ee.ImageCollection('COPERNICUS/S1_GRD')\
                .filterBounds(jedeb)\
                .filter(season)\
                .filter(ee.Filter.eq('instrumentMode', 'IW'))\
                .filter(ee.Filter.eq('resolution_meters', 10)) 
                
# Number of tiles in the season
sentinel1 = sentinel1.sort('system:time_start')
sentinel1.size().getInfo()

13

In [29]:
# the acquisition times in the collection, formatted with Python's time module
acq_times = sentinel1.aggregate_array('system:time_start').getInfo()
[time.strftime('%x', time.gmtime(acq_time/1000)) for acq_time in acq_times]

['12/02/21',
 '12/14/21',
 '12/26/21',
 '01/07/22',
 '01/19/22',
 '01/31/22',
 '02/12/22',
 '02/24/22',
 '03/08/22',
 '03/20/22',
 '04/01/22',
 '04/13/22',
 '04/25/22']

In [30]:
# using monthly mean backscatter 

s1Dec = ee.Image(sentinel1.filterDate("2021-12-01","2021-12-31").mean()).select(["VV","VH"],["VVdec","VHdec"])
s1Jan = ee.Image(sentinel1.filterDate("2022-01-01","2022-01-31").mean()).select(["VV","VH"],["VVjan","VHjan"]);
s1Feb = ee.Image(sentinel1.filterDate("2022-02-01","2022-02-28").mean()).select(["VV","VH"],["VVfeb","VHfeb"])
s1Mar = ee.Image(sentinel1.filterDate("2022-03-01","2022-03-31").mean()).select(["VV","VH"],["VVmar","VHmar"])
s1Apr = ee.Image(sentinel1.filterDate("2022-04-01","2022-04-30").mean()).select(["VV","VH"],["VVapr","VHapr"])

sentinel1mean = s1Dec.addBands(s1Jan)\
                 .addBands(s1Feb)\
                 .addBands(s1Mar)\
                 .addBands(s1Apr)

#print(sentinel1mean.getInfo())

In [31]:
composite = ee.Image.cat([(s2median),(sentinel1mean)])
sentinel = composite.clip(jedeb)
#print(Sentinel.getInfo())

## Train the Classifier

In [32]:
bands = ['B2', 'B3', 'B4','B8', 'B8A', 'B11', 'B12', 'VVdec', 'VHdec', 'VVjan', 'VHjan', 'VVfeb', 'VHfeb', 'VVmar', 'VHmar', 'VVapr', 'VHapr']
# This property of the table stores the land cover labels.
label = 'Cover'

# Overlay the points on the imagery to get training.
sample = sentinel.select(bands).sampleRegions(
    **{'collection': points, 'properties': [label], 'scale': 10}
)

In [33]:
classifier = ee.Classifier.smileCart(128).train(sample, label, bands)

## Classify the Image

In [34]:
# Classify the image with the same bands used for training.
classified_RF2 = sentinel.select(bands).classify(classifier)

## Accuracy assessment

Training Accuracy

In [35]:
train_accuracy = classifier.confusionMatrix()
#train_accuracy.getInfo()

In [36]:
#train_accuracy.accuracy().getInfo()

In [37]:
#train_accuracy.kappa().getInfo()

In [38]:
#train_accuracy.producersAccuracy().getInfo()

In [39]:
#train_accuracy.consumersAccuracy().getInfo()

Validation Accuarcy

In [40]:
# This property of the table stores the land cover labels.
label = 'Cover'

band = ['classification']
# Overlay the points on the imagery to get training.
test = classified_RF2.select(band).sampleRegions(
    **{'collection': validationGcp, 'properties':[label],'scale': 10}
)

In [41]:
test_accuracy = test.errorMatrix('Cover', 'classification')
#test_accuracy.getInfo()

In [42]:
test_accuracy.accuracy().getInfo()

0.8177777777777778

In [43]:
test_accuracy.kappa().getInfo()

0.7669512934518998

In [44]:
test_accuracy.producersAccuracy().getInfo()

[[0],
 [1],
 [0.8285714285714286],
 [0.6470588235294118],
 [0.9090909090909091],
 [0.7924528301886793],
 [1],
 [0.7777777777777778]]

In [45]:
test_accuracy.consumersAccuracy().getInfo()

[[0,
  1,
  0.8529411764705882,
  0.7857142857142857,
  0.8333333333333334,
  0.75,
  0.8571428571428571,
  0.875]]

# Improving Classification 

## Vegetation Indices 

EVI

In [46]:
def getEVI(image):
    # Compute the EVI using an expression.
    EVI = image.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
            'NIR': image.select('B8'),
            'RED': image.select('B4'),
            'BLUE': image.select('B2')
        }).rename("EVI")

    image = image.addBands(EVI)

    return(image)

evi = sentinel2.map(getEVI).median().clip(jedeb).select('EVI')

print(evi.getInfo())

{'type': 'Image', 'bands': [{'id': 'EVI', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


NDVI

In [83]:
# Calculate NDVI
def getNDVI(image):
    # Compute the EVI using an expression.
    NDVI = image.expression(
        '((NIR - RED) / (NIR + RED ))', {
            'NIR': image.select('B8'),
            'RED': image.select('B4'),
        }).rename("NDVI")

    image = image.addBands(NDVI)

    return(image)

ndvi = sentinel2.map(getNDVI).select('NDVI')

#print(ndvi.getInfo())

In [108]:
# Filter the first Sentinel 2 image of the month
sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR')\
    .filterBounds(jedeb)\
    .filter(season)\
    .map(maskS2clouds)

# Number of tiles in the season
sentinel2 = sentinel2.sort('system:time_start')
sentinel2.size().getInfo()

30

NDBI

In [48]:
# Calculate NDBI
def getNDBI(image):
    # Compute the EVI using an expression.
    NDBI = image.expression(
        '((SWIR - NIR) / (SWIR + NIR ))', {
            'SWIR': image.select('B11'),
            'NIR': image.select('B8'),
        }).rename("NDBI")

    image = image.addBands(NDBI)

    return(image)

ndbi = sentinel2.map(getNDBI).median().clip(jedeb).select('NDBI')

print(ndbi.getInfo())

{'type': 'Image', 'bands': [{'id': 'NDBI', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


Slope

## Auxillary Data

In [49]:
dem = ee.Image('NASA/NASADEM_HGT/001').select('elevation').clip(jedeb)

In [50]:
slope = ee.Terrain.slope(dem)
Map.addLayer(slope, {min: 0, max: 89.99}, 'Slope')
print(slope.getInfo())

{'type': 'Image', 'bands': [{'id': 'slope', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': 0, 'max': 90}, 'crs': 'EPSG:4326', 'crs_transform': [0.0002777777777777778, 0, -179.0001388888889, 0, -0.0002777777777777778, 61.00013888888889]}]}


Composite

In [51]:
composite = ee.Image.cat([(sentinel),(evi),(ndvi),(ndbi),(slope)])

sentinel_vi = composite.clip(jedeb)

## Train the classifier 

In [52]:
bands = ['B2', 'B3', 'B4','B8', 'B8A', 'B11', 'B12',\
         'VVdec', 'VHdec', 'VVjan', 'VHjan', 'VVfeb', 'VHfeb', 'VVmar', 'VHmar', 'VVapr', 'VHapr',\
         'EVI','NDBI','slope']
# This property of the table stores the land cover labels.
label = 'Cover'

# Overlay the points on the imagery to get training.
sample = sentinel_vi.select(bands).sampleRegions(
    **{'collection': points, 'properties': [label], 'scale': 10}
)

In [53]:
classifier = ee.Classifier.smileCart(128).train(sample, label, bands)

## Classify the Image

In [54]:
# Classify the image with the same bands used for training.
classified_RF3 = sentinel_vi.select(bands).classify(classifier)

## Accuracy Assessment

Training Accuarcy

In [55]:
train_accuracy = classifier.confusionMatrix()
#train_accuracy.getInfo()

In [56]:
#train_accuracy.accuracy().getInfo()

In [57]:
#train_accuracy.kappa().getInfo()

In [58]:
#train_accuracy.producersAccuracy().getInfo()

In [59]:
#train_accuracy.consumersAccuracy().getInfo()

Validation Accuarcy

In [60]:
# This property of the table stores the land cover labels.
label = 'Cover'

band = ['classification']
# Overlay the points on the imagery to get training.
test = classified_RF3.select(band).sampleRegions(
    **{'collection': validationGcp, 'properties':[label],'scale': 10}
)

In [61]:
test_accuracy = test.errorMatrix('Cover', 'classification')
#test_accuracy.getInfo()

In [62]:
test_accuracy.accuracy().getInfo()

0.7955555555555556

In [63]:
test_accuracy.kappa().getInfo()

0.7383720930232558

In [64]:
test_accuracy.producersAccuracy().getInfo()

[[0],
 [1],
 [0.7857142857142857],
 [0.6470588235294118],
 [0.9318181818181818],
 [0.7924528301886793],
 [0.8333333333333334],
 [0.6666666666666666]]

In [65]:
test_accuracy.consumersAccuracy().getInfo()

[[0,
  1,
  0.8208955223880597,
  0.88,
  0.7592592592592593,
  0.7636363636363637,
  0.8333333333333334,
  0.6666666666666666]]

# Visualize Results

In [66]:
# Define a palette for the classification.
Palette = [
    '419BDF', #water
    '397D49', #trees
    '88B053', #grass
    '6efc6a', #irrigation
    'E49635', #crops
    'C4281B', #built
    'A59B8F', #bare
]
Map.addLayer(classified_RF3, {'palette': Palette, 'min': 1, 'max': 7}, 'classification')
Map

Map(center=[10.41, 37.62], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(childre…

# Calculate Statistics

In [67]:
# Calculate total area of a single class (eq(*))     * is the class number 
total_area = classified_RF1.multiply(ee.Image.pixelArea()).reduceRegion(**{
  'reducer': ee.Reducer.sum(),
  'geometry': jedeb.geometry(),
  'scale': 10,
  'maxPixels': 1e9
})

total_area.getInfo()

{'classification': 3378278839.3644485}

In [68]:
# Calculate total area of a single class (eq(*))     * is the class number 
area = classified_RF1.eq(4).multiply(ee.Image.pixelArea()).reduceRegion(**{
  'reducer': ee.Reducer.sum(),
  'geometry': jedeb.geometry(),
  'scale': 10,
  'maxPixels': 1e9
})

area.getInfo()

{'classification': 85415740.12398869}

In [69]:
# Calculate total area of a single class (eq(*))     * is the class number 
area = classified_RF2.eq(4).multiply(ee.Image.pixelArea()).reduceRegion(**{
  'reducer': ee.Reducer.sum(),
  'geometry': jedeb.geometry(),
  'scale': 10,
  'maxPixels': 1e9
})

print(area.getInfo(),'Area in ha')

{'classification': 66030907.97929546} Area in ha


In [70]:
# Calculate total area of a single class (eq(*))     * is the class number 
area = classified_RF3.eq(4).multiply(ee.Image.pixelArea()).reduceRegion(**{
  'reducer': ee.Reducer.sum(),
  'geometry': jedeb.geometry(),
  'scale': 10,
  'maxPixels': 1e9
})

area.getInfo()

{'classification': 63109723.581111856}